## Options Backtesting Framework

This is a simple backtesting framework for options. Given end of day data for both the capital and derivatives markets, we would be backtesting for different strategies.

> All options are opened at the start of the period and closed by the end of the period or by a stop loss if provided

### Implementation

1. Load and transform necessary data
2. Create a list of option contracts to be entered into each day
3. Create a list of signals on, whether to enter into the contract, for each day
4. Extract the necessary data for the options contracts that would be executed
5. Analyze Profit and loss and other metrices


In [9]:
# Load necessary libraries

import pandas as pd
import numpy as np
from fastbt.rapid import backtest

## Utility Functions

In [25]:
def itm(price, opt='C', step=100):
    """
    Get the strike price of the in the money option
    price
        spot price
    opt
        option type - C for Call and P for Put
    step
        multiple of option strike price
    """
    if opt == 'C':
        return int(price/step)*step
    elif opt == 'P':
        return (int(price/step)+1)*step

Data Preparation
-----------------
Load the raw data and do some transformations. All raw options data is loaded from the bhav copy of the NSE website. Index data is from [niftyindices.com](https://www.niftyindices.com/).

In [94]:
nifty = pd.read_csv('/home/machine/Downloads/NIFTY 50_Data.csv',
                   parse_dates=['Date'])
nifty.rename(lambda x: x.lower(), axis='columns', inplace=True)
nifty = nifty[nifty.date.dt.year==2019].sort_values(by='date').reset_index(drop=True)

der = pd.read_hdf('/media/machine/4E1EA2D152455460/data/output/fno_2019.h5')
der.rename({'timestamp': 'date'}, axis='columns', inplace=True)
der['exp_day'] = der.expiry_dt.dt.day

# Difference between expiry date and date
der['day_diff'] = (der['expiry_dt'] - der['date']).dt.days


In [101]:
# Change your symbol here
symbol = 'NIFTY'
# Keep nifty data with monthly options only
# This hack is incomplete and should be correctly configured
q = '(symbol=="{sym}") & (instrument=="OPTIDX") & (exp_day > 24) & (day_diff<31)'
df = der.query(q.format(sym=symbol))
pd.Series(df.expiry_dt.unique()).sort_values()

2   2019-01-31
1   2019-02-28
3   2019-03-28
0   2019-04-25
4   2019-05-30
5   2019-06-27
dtype: datetime64[ns]

Select list of options
-----------------------

We must now select the list of options to be traded. To begin, with let's BUY in the money call option at the start of the day and hold it till the end of the day. We need the following columns for each day

 * date
 * strike price
 * option_type
 * spot price

In [103]:
nifty['strike'] = [itm(x) for x in nifty.open]
nifty['spot'] = nifty.open.copy()
nifty['opt'] = 'CE'
cols = ['date', 'strike', 'opt', 'spot']
nifty[cols].head()

,date,strike,opt,spot
0,2019-01-01,10800,CE,10881.70
1,2019-01-02,10800,CE,10868.85
2,2019-01-03,10700,CE,10796.80
3,2019-01-04,10600,CE,10699.70
4,2019-01-07,10800,CE,10804.85
